Output dataset must have the following format:

[[‘Who is Shaka Khan?’, {‘entities’: [[7, 17, ‘PERSON’]]}],
 [‘I like London and Berlin.’,
  {‘entities’: [[7, 13, ‘LOC’], [18, 24, ‘LOC’]]}]]
  
  
#### Credits: 

https://dfelo93.medium.com/how-to-train-a-spacy-model-for-ner-cc8f964372ca

In [1]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.cli.train import train
import json
import random

In [ ]:
sentences = []
with open("../../Data/NER_annotated_data/chatgpt.txt", "r", encoding = "utf-8") as f:
    for line in f:
        if ', {"entities' not in line: continue
        end_index = line.index(', {"entities')
        end_index -= 1
        start_index = line.index('"')
        start_index += 1
        sentence = line[start_index : end_index]
        print(sentence)
        sentences.append(sentence)

In [ ]:
with open("../../Data/NER_annotated_data/chatgpt_sentences.txt", "w", encoding = "utf-8") as f:
    for i, sentence in enumerate(sentences):
        f.write(sentence)
        if i == len(sentences) - 1: continue
        f.write("\n")

In [ ]:
# Converting JSON dataset to spaCy docbin binary file. DOES NOT WORK. Deprecated in Spacy 3X
# !python -m spacy convert ./../../Data/NER_annotated_data/ChatGPT/annotations.json ./../../Data/NER_annotated_data/ChatGPT/

In [2]:
# Converting JSON dataset manually into spaCy doc bin format
with open("./../../Data/NER_annotated_data/ChatGPT/annotations.json", "r") as f:
    data = json.load(f)

In [3]:
nlp = spacy.blank("en")
doc_bin = DocBin()

In [4]:
for i in range(len(data['annotations'])):
    if not data['annotations'][i]: continue
    text, annotations = data['annotations'][i]
    doc = nlp.make_doc(text.strip())
    ents = []
    for start, end, label in annotations["entities"]:
        span = doc.char_span(start, end, label=label)
        if span is None:
            msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
            warnings.warn(msg)
        else:
            ents.append(span)
    doc.ents = ents
    doc_bin.add(doc)

In [6]:
doc_bin.to_disk("./../../Data/NER_annotated_data/ChatGPT/annotations.spacy")